<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#加载库文件" data-toc-modified-id="加载库文件-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>加载库文件</a></span></li><li><span><a href="#构建数据" data-toc-modified-id="构建数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>构建数据</a></span></li><li><span><a href="#把数据写入TFRecords文件中" data-toc-modified-id="把数据写入TFRecords文件中-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>把数据写入TFRecords文件中</a></span></li><li><span><a href="#读取TFRecords文件" data-toc-modified-id="读取TFRecords文件-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>读取TFRecords文件</a></span></li></ul></div>

# 加载库文件

In [6]:
# 加载库文件
import tensorflow as tf
import numpy as np

# 构建数据

In [19]:
# 数据形式
data = [{'name': b"xiaoming",
         'age': 7,
         'height': 120.5,
         'score': [89.0, 91.0, 72.0],
         'pic': [[1, 2, 3], [4, 5, 6], [7, 8, 9]]},
        {'name': b"xiaohong",
         'age': 8,
         'height': 130.8,
         'score': [96.0, 85.2, 76.5],
         'pic': [[10, 11, 12], [13, 14, 15], [16, 17, 18]]},
        {'name': b"xiaozhang",
         'age': 9,
         'height': 140.3,
         'score': [96.7, 76.8, 68.9],
         'pic': [[19, 20, 21], [22, 23, 24], [25, 26, 27]]}]

# 把pic转化成图片数据格式
data[0]['pic'] = np.array(data[0]['pic']).astype(np.uint8)
data[1]['pic'] = np.array(data[1]['pic']).astype(np.uint8)
data[2]['pic'] = np.array(data[2]['pic']).astype(np.uint8)

# 把数据写入TFRecords文件中

In [21]:
# 申请写入example的writer
writer = tf.python_io.TFRecordWriter("data.tfrecords")

for index in range(3):
    # 获取每一行的数据
    name = data[index]['name']
    age = data[index]['age']
    height = data[index]['height']
    score = data[index]['score']
    pic = data[index]['pic']
    
    # 1.把数据提取为message Feature，即属性值
    bytes_name = tf.train.Feature(bytes_list=tf.train.BytesList(value=[name]))
    int64_age = tf.train.Feature(int64_list=tf.train.Int64List(value=[age]))
    float_height = tf.train.Feature(float_list=tf.train.FloatList(value=[height]))
    # 因为score本身就是列表, 不需要加[]
    float_score = tf.train.Feature(float_list=tf.train.FloatList(value=score))
    # 因为pic是图像数据，先要对pic进行字符化
    pic_raw = pic.tostring()
    bytes_pic = tf.train.Feature(bytes_list=tf.train.BytesList(value=[pic_raw]))
    
    # 2.构建message Features
    feature = {
        'name': bytes_name,
        'age': int64_age,
        'height': float_height,
        'score': float_score,
        'pic': bytes_pic}
    features = tf.train.Features(feature=feature)
    
    # 3.构建Example Protocol Buffer数据
    example = tf.train.Example(features=features)
    
    # 4.序列化
    serialized = example.SerializeToString()
    
    # 把序列化数据写入tfrecord文件
    writer.write(serialized)
    
# 关闭writer
writer.close()

# 读取TFRecords文件

In [28]:
# 1.申请一个读取TFRecords对象
reader = tf.TFRecordReader()

# 2.创建一个队列维护输入文件的管道
file_queue = tf.train.string_input_producer(['data.tfrecords'])

# 读取一个序列化的example
# 也可以使用read_up_to函数一次性读取很多个样例
_, serialized_example = reader.read(file_queue)

# 3.解析读入的一个样例，使用函数parse_single_example。
# 如果要解析多个样例，可以使用函数parse_example。
name_bytes = tf.FixedLenFeature([], tf.string)
age_int64 = tf.FixedLenFeature([], tf.int64)
height_float = tf.FixedLenFeature([], tf.float32)
score_float = tf.FixedLenFeature([3], tf.float32)
pic_raw = tf.FixedLenFeature([], tf.string)

features = tf.parse_single_example(
    serialized_example,
    features={
        'name': name_bytes,
        'age': age_int64,
        'height': height_float,
        'score': score_float,
        'pic': pic_raw,
    })

# 4.将数据转化为所需的类型
name = tf.cast(features['name'], tf.string) # name本身就是tf.string类型
age = tf.cast(features['age'], tf.int32)
height = tf.cast(features['height'], tf.float32)
score = tf.cast(features['score'], tf.float32)
pic = tf.decode_raw(features['pic'], tf.uint8) # 按tf.uint8进行解码
pic = tf.reshape(pic, shape=[3, 3]) # 调整pic为所需要的结构

# 建立多线程
coord = tf.train.Coordinator()

with tf.Session() as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    # 启动线程
    threads = tf.train.start_queue_runners(sess, coord=coord)
    
    # 读取样例中的数据
    # 每运行一次，读取一次样例
    for index in range(3):
        name_data, age_data, height_data, score_data, pic_data = sess.run([name, age, height, score, pic])
        print("name: ", name_data)
        print("age: ", age_data)
        print("height: ", height_data)
        print("score: ", score_data)
        print("pic: \n", pic_data)
        print("\n")
    

name:  b'xiaoming'
age:  7
height:  120.5
score:  [ 89.  91.  72.]
pic: 
 [[1 2 3]
 [4 5 6]
 [7 8 9]]


name:  b'xiaohong'
age:  8
height:  130.8
score:  [ 96.          85.19999695  76.5       ]
pic: 
 [[10 11 12]
 [13 14 15]
 [16 17 18]]


name:  b'xiaozhang'
age:  9
height:  140.3
score:  [ 96.69999695  76.80000305  68.90000153]
pic: 
 [[19 20 21]
 [22 23 24]
 [25 26 27]]


INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer_6/input_producer_6_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer_6, input_producer_6/RandomShuffle)]]
